## Import required packages

In [0]:
# Download required packages
!pip -q install gdown missingno torch petastorm

%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window

from petastorm.spark import SparkDatasetConverter, make_spark_converter
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/petastorm/cache')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import torch
import re
import torch.nn as nn
from typing import *
import datetime
import gdown

import tqdm as tq
def tqdm(*args, **kwargs):
  ''' Small trick to prevent tqdm printing newlines at each step. '''
  return tq.tqdm(*args, **kwargs, leave=True, position=0)

WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

## Data aquisition
We retrieve our datasets and download them to a temporary directory in the driver node.

In [0]:
!rm -rf /tmp/data /tmp/__MACOSX
gdown.download('https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM', '/tmp/data.zip', quiet=False)
!unzip -q /tmp/data.zip -d /tmp/
!rm /tmp/data.zip

Downloading...
From: https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM
To: /tmp/data.zip
0.00B [00:00, ?B/s]8.91MB [00:00, 87.9MB/s]27.8MB [00:00, 146MB/s] 46.7MB [00:00, 165MB/s]65.0MB [00:00, 171MB/s]71.9MB [00:00, 162MB/s]

Then we load the datasets to the DBFS.

In [0]:
dbutils.fs.mv("file:/tmp/data", "dbfs:/data", recurse=True)

Out[16]: True

In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


## Dataset loading

In [0]:
key_stats_df = spark.read.load("dbfs:/data/key_stats_yahoo.csv", 
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true"
                          )

# Drop the first ID column
key_stats_df = sc.parallelize(key_stats_df.drop(key_stats_df.columns[0]).head(1005)).toDF()#TODO: remove head(n) (only meant for development)
#key_stats_df = key_stats_df.drop(key_stats_df.columns[0])
key_stats_df.schema['Date'].nullable = False

# Use legacy format to parse dates
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
key_stats_df = key_stats_df.withColumn("Date", F.to_date(key_stats_df["Date"], 'MM/dd/yyyy HH:mm'))

# Cast numerical columns to double
for column in key_stats_df.columns[2:]:
  key_stats_df = key_stats_df.withColumn(column, key_stats_df[column].cast("double"))

# Prices dataframes for every stock #TODO: remove :N (only meant for development)
prices_files = [f.path for f in dbutils.fs.ls('/data/prices/')[:10] if f.path.endswith('.csv')]
#prices_files = [f.path for f in dbutils.fs.ls('/data/prices/') if f.path.endswith('.csv')]
dfs_names = [f.rsplit('/', 1)[1][:-len('.csv')] for f in prices_files]
prices_dfs = []
for f in tqdm(prices_files, desc='Reading stock price data', total=len(prices_files)):
  df = spark.read.load(f,
                       format="csv",
                       sep=",",
                       inferSchema="true",
                       header="true"
                      )
  df = df.withColumn("Date", F.to_date(df["Date"], 'dd-MM-yyyy'))
  df.schema['Date'].nullable = False
  prices_dfs.append(df)

Reading stock price data: 0%| | 0/9 [00:00<?, ?it/s]Reading stock price data: 11%|█ | 1/9 [00:00<00:06, 1.30it/s]Reading stock price data: 22%|██▏ | 2/9 [00:01<00:05, 1.24it/s]Reading stock price data: 33%|███▎ | 3/9 [00:02<00:04, 1.21it/s]Reading stock price data: 44%|████▍ | 4/9 [00:03<00:04, 1.22it/s]Reading stock price data: 56%|█████▌ | 5/9 [00:04<00:03, 1.26it/s]Reading stock price data: 67%|██████▋ | 6/9 [00:04<00:02, 1.20it/s]Reading stock price data: 78%|███████▊ | 7/9 [00:05<00:01, 1.21it/s]Reading stock price data: 89%|████████▉ | 8/9 [00:06<00:00, 1.21it/s]Reading stock price data: 100%|██████████| 9/9 [00:07<00:00, 1.25it/s]Reading stock price data: 100%|██████████| 9/9 [00:07<00:00, 1.23it/s]

## Dataset analysis

In [0]:
print("Prices dataframe format:")
prices_dfs[0].printSchema()

Prices dataframe format:
root
-- Date: date (nullable = true)
-- Low: double (nullable = true)
-- Open: double (nullable = true)
-- Volume: integer (nullable = true)
-- High: double (nullable = true)
-- Close: double (nullable = true)
-- Adjusted Close: double (nullable = true)

In [0]:
print("Key stats dataframe format:")
key_stats_df.printSchema()

Key stats dataframe format:
root
-- Date: date (nullable = true)
-- Ticker: string (nullable = true)
-- Price: double (nullable = true)
-- DE Ratio: double (nullable = true)
-- Trailing P/E: double (nullable = true)
-- Price/Sales: double (nullable = true)
-- Price/Book: double (nullable = true)
-- Profit Margin: double (nullable = true)
-- Operating Margin: double (nullable = true)
-- Return on Assets: double (nullable = true)
-- Return on Equity: double (nullable = true)
-- Revenue Per Share: double (nullable = true)
-- Market Cap: double (nullable = true)
-- Enterprise Value: double (nullable = true)
-- Forward P/E: double (nullable = true)
-- PEG Ratio: double (nullable = true)
-- Enterprise Value/Revenue: double (nullable = true)
-- Enterprise Value/EBITDA: double (nullable = true)
-- Revenue: double (nullable = true)
-- Gross Profit: double (nullable = true)
-- EBITDA: double (nullable = true)
-- Net Income Avl to Common : double (nullable = true)
-- Diluted EPS: double (nullable = true)
-- Earnings Growth: double (nullable = true)
-- Revenue Growth: double (nullable = true)
-- Total Cash: double (nullable = true)
-- Total Cash Per Share: double (nullable = true)
-- Total Debt: double (nullable = true)
-- Current Ratio: double (nullable = true)
-- Book Value Per Share: double (nullable = true)
-- Cash Flow: double (nullable = true)
-- Beta: double (nullable = true)

### Utility functions

In [0]:
# TODO: add remaining utility functions

def prices_df_nan_summary(prices_dfs: List[pyspark.sql.DataFrame], names: List[str]) -> pd.DataFrame:
  ''' Utility function to summarize columns that have missing values. '''
  nan_dfs = []
  for prices_df, name in tqdm(zip(prices_dfs, names), total=len(prices_dfs), desc='Generating prices summary ...'):
    nan_absolute = prices_df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in prices_df.columns]).first()
    if any(nan_absolute):
      # Simple conversion from Pyspark row -> Python set of values
      values = set(nan_absolute.asDict().values()).difference({0})
      # Either we don't have values for that row, or we have all of them (but Date which is non-nullable)
      # Values contains the no. of NaN values and 0 in correspondance of the Date column
      assert len(values) == 1
      nan_count = values.pop()
      nan_dfs.append((name, round(100*nan_count/prices_df.count(), 3), nan_count))

  return pd.DataFrame(nan_dfs, columns=['Stock name', 'Missing data (%)', 'Count'])

def remove_trailing_nan(df: pyspark.sql.DataFrame, ticker: str, col: str = 'Low') -> pyspark.sql.DataFrame:
  '''
    A trick to detect if the input DataFrame ends with a contiguous collection of NaN rows, returns the dataframe without them.
  '''
  # The total number of rows of the dataframe
  df_length = df.count()
  
  # Sort the input dataframe and add a new column to keep track of the relative position of each row
  df_sorted_id = df.sort('Date').withColumn('id', F.row_number().over(Window.orderBy('Date')))
  
  # Tricky part here: create a new column called 'cumsum' that will store the progressive number of consecutive NaN in our dataset.
  # Let's break it into smaller parts:
  # 1) create an index generator that will partition by 'Low' values [(...,Null...), (...,value1,...), (...,value2...), ... (...)] and within rows order by date
  # Example:
  ## |2019-06-05|null|null|  null|null| null|          null|624|        21|
  ## |2019-06-06|null|null|  null|null| null|          null|625|        22|
  ## |2019-06-07|null|null|  null|null| null|          null|626|        23| <- last column is the cumulative sum (i.e. the number of consecutive NaN)
  ## ...
  ## |2019-05-09|23.2|11.5|   7.3|4.2|  16.2|          29.1|1  |         0|
  ## |2019-05-10|23.2|11.5|   7.3|4.2|  16.2|          29.1|2  |         0|
  # 2) assign to each row a progressive index starting from 1 if it has null in correspondance of Low, zero otherwise
  # 3) store these values into a new column called cumsum (i.e. it behaves like pandas cumsum)
  # 4) at the end, the row whose ID corresponds to the length of the dataframe will contain at column 'cumsum' the no. of trailing NaN values.
  cumsum_df = df_sorted_id.withColumn('cumsum', F.when(F.isnull(df_sorted_id.Low), F.row_number().over(Window.partitionBy('Low').orderBy('Date'))).otherwise(0))

  # Retrieve the "last" row and read the value of cumsum
  end_idx = cumsum_df.where(cumsum_df['id'] == df_length).first().cumsum
  
  # Retain rows whose index is lower len(df) - end_idx + 1 (i.e. cut trailing NaN values)
  return df_sorted_id.where(df_sorted_id['id'] <= df_length-end_idx+1)


def merge_prices_fundamentals(
    prices_dfs: List[pyspark.sql.DataFrame],
    key_stats_df: pyspark.sql.DataFrame,
    dfs_names: List[str],
    drop_cols: List[str] = ['Date', 'Ticker', 'Price']
    ) -> List[pyspark.sql.DataFrame]:
  # Define the target list of dataframes
  prices_dfs_new = []
  for ticker in tqdm(key_stats_df.select('Ticker').distinct().collect(), desc='Merging the datasets ...'):
    ticker = ticker[0]
    # Consider only stocks for which we have fundamental data
    if ticker.upper() not in dfs_names: continue
      
    # The prices dataframe associated to the current ticker
    prices_df_idx = dfs_names.index(ticker.upper())
    prices_df = prices_dfs[prices_df_idx]
    
    # Retrieve financial reports for the current ticker
    ticker_df = key_stats_df.filter(F.col('Ticker') == ticker)
    
    # Perform an inner join between the two dataframes, we first take all reports
    # whose date is at most the prices date (then we will take the latest available)
    joined_df = ticker_df.join(prices_df, ticker_df.Date <= prices_df.Date, how='inner')
    
    # Let's break this into smaller parts:
    # 1) Add a new column for the date latest available report for that stock. To do that
    #    we partition over **prices** Date using the Window object, and the the max over
    #    **ticker** Date.
    # 2) Retain those columns whose financial report date is the same as in ticker_df_max_date
    # 3) Finally drop the **ticker** Date and other unused columns
    joined_df = joined_df.withColumn('ticker_df_max_date', F.max(ticker_df['Date'])\
                      .over(Window.partitionBy(prices_df['Date'])))\
                      .where(ticker_df['Date'] == F.col('ticker_df_max_date'))\
                      .drop('ticker_df_max_date', 'High', 'Low', 'Open', 'Close')\
                      .drop(ticker_df['Date'])\
                      
    # Return the usual list of dataframes
    prices_dfs_new.append(joined_df)
    
  return prices_dfs_new
    
                
def fill_missing_days(
  aggregate_dfs: List[pyspark.sql.DataFrame],
  remove_weekends: bool = True,
  end_date: str = '2014-01-01'
) -> List[pyspark.sql.DataFrame]:
  result_dfs = []
  
  @udf("boolean")
  def is_weekday(date: datetime) -> bool:
      ''' Returns true if the provided date corresponds to a weekday. '''
      return date.weekday() < 5
    
  for df in tqdm(aggregate_dfs, desc='Filling missing days ...'):
    # In this case we apply the following steps in order to apply fill-forward to our dataset:
    # 1) define a Window object the will over the dataset sorted with decreasing dates
    # 2) compute the difference in days between consecutive rows (11/05/2021 - 07/05/2021 ==> 4 days)
    #    and store it in a column named diff
    # 3) compute an increasing index in the column seq that is the days gap we need to fill
    # Example:
    # |Date      | Adjusted close   |diff|seq|new_date  |
    # |2021-04-09|116.80999755859375|   3|  1|2021-04-11|
    # |2021-04-09|116.80999755859375|   3|  2|2021-04-10|
    # |2021-04-09|116.80999755859375|   3|  3|2021-04-09|
    # 4) create a new column new_date as the result of the computation F.col('Date') + F.col('diff') - F.col('seq')
    # 5) only retain rows whose date is before the provided end_date
    #
    # N.B. if we want to bfill the values we should instead consider the dataframes with dates in ascending order,
    # and subtract the value of F.col('diff') instead of summing it. Other parts of the code stays the same.
    df = df\
    .withColumn('diff', F.datediff(F.lag(F.col('Date'),1).over(Window.orderBy(F.desc('Date'))), F.col('Date')))\
    .withColumn('seq', F.explode(F.sequence(F.lit(1), F.col("diff"))))\
    .withColumn('new_date', (F.col('Date') + F.col('diff') - F.col('seq')))\
    .where(F.col('new_date') < pd.Timestamp(end_date))
    
    # If specified, remove weekends that are most likely created by us as synthetic data with the ffill technique
    if remove_weekends:
      df = df.where(is_weekday(F.col('new_date')))
      
    # Drop unecessary columns
    df = df.drop('Date', 'diff', 'seq').withColumnRenamed('new_date', 'Date')
    
    # If the dataframe is actually non-empty, add it to the list of resulting dataframes
    if df.count() > 0:
      result_dfs.append(df)
  return result_dfs

def missing_values_summary(df):
  ''' Returns a utility summary to view missing values in our dataframe. '''
  n = df.count()
  
  def to_percentage(x: pyspark.sql.column.Column, n: int) -> int:
    ''' Utility function to compute the amount of missing values as a percentage of the original dataframe. '''
    return F.round(100 * x / n, 3)
  
  # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe
  return df.agg(*[to_percentage(F.count(F.when(F.isnull(c), c)), n).alias(c) for c in df.columns]).first()

def scale_features(dfs: List[pyspark.sql.DataFrame]) -> None:
    ''' Scales the numerical features to unit variance. '''
    
    scalable_features = ['DE Ratio', 'Trailing P/E', 'Price/Sales', 'Price/Book',
       'Profit Margin', 'Operating Margin', 'Return on Assets',
       'Return on Equity', 'Revenue Per Share', 'Market Cap',
       'Enterprise Value', 'PEG Ratio', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Revenue', 'Gross Profit', 'EBITDA',
       'Net Income Avl to Common ', 'Diluted EPS', 'Earnings Growth',
       'Revenue Growth', 'Total Cash', 'Total Cash Per Share', 'Total Debt',
       'Current Ratio', 'Book Value Per Share', 'Cash Flow', 'Beta', 'Volume',
       'Adjusted Close', 'SMA', 'RSI']
    

    aggregate_df = dfs[0]
    for df in dfs[1:]:
      aggregate_df = aggregate_df.union(df)
      
    mean = aggregate_df.select(
      [F.mean(F.col(column)) for column in scalable_features]
    ).collect()[0]
    
    std = aggregate_df.select(
      [F.stddev(F.col(column)) for column in scalable_features]
    ).collect()[0]

    
    for i in tqdm(range(len(dfs)), desc='Scaling numerical features ...'):
      # Scaling scalable features
      for j, feature in enumerate(scalable_features):
        dfs[i] = dfs[i].withColumn(feature, (dfs[i][feature]-mean[j])/std[j])
        
def impute_missing_values(
  prices_dfs_new: List[pyspark.sql.DataFrame],
  key_stats_df: pyspark.sql.DataFrame
) -> Tuple[List[pyspark.sql.DataFrame], pyspark.sql.DataFrame]:
  # define the window
  window = Window.orderBy('Date').rowsBetween(Window.unboundedPreceding, 0)

  # Forward filling values 
  # (ref. https://stackoverflow.com/questions/38131982/forward-fill-missing-values-in-spark-python/50422240#50422240)
  for i in range(len(prices_dfs_new)):
    for col_name in prices_dfs_new[i].schema.names:
      col = F.last(prices_dfs_new[i][col_name], ignorenulls=True).over(window)
      prices_dfs_new[i] = prices_dfs_new[i].withColumn(col_name, col)

  # In this case this dataframe contains financial reports that may contain NaN values either because that
  # metric was not available at that time OR because it was monitoring an initial stage of a company growth.
  # What we do is to apply the classic fast-forward, and fill initial missing values with zeroes.
  # Please note: we also discard the 'Forward P/E' column since the imputation here would introduce too much noise.
  key_stats_df_new = key_stats_df.drop('Forward P/E')
  for col_name in key_stats_df_new.schema.names:
      col = F.last(key_stats_df_new[col_name], ignorenulls=True).over(window)
      key_stats_df_new = key_stats_df_new.withColumn(col_name, col)
  key_stats_df_new = key_stats_df_new.fillna(0.)
  
  return prices_dfs_new, key_stats_df_new

def from_dfs(
  dfs: List[pyspark.sql.DataFrame],
  window_size: int = 3,
  target_col: str = 'Adjusted Close',
  val_date: str = '2012-01-01',
  test_date: str = '2013-01-01'
):
  # Split the dataset into train/val/test using these dates as boundaries
  val_begin = pd.Timestamp(val_date)
  test_begin = pd.Timestamp(test_date)
  
  def should_drop(col: pyspark.sql.column, idx: int) -> bool:
    ''' Returns whether the provided column should be dropped. '''
    return (col.endswith(f'_{idx}') or 'Ticker' in col or 'Date' in col or 'id' in col)\
              and col != f'Adjusted Close_{idx}'

  train_split, val_split, test_split = [], [], []
  
  for df in tqdm(dfs):
    # Create an ID for each row based on dates
    df = df\
      .sort('Date')\
      .withColumn('id', F.row_number().over(Window.orderBy('Date')))
    
    # Split the stock dataset into train/dev/test splits
    train_df = df.where(F.col('Date') < val_begin)
    val_df = df.where(F.col('Date').between(val_begin, test_begin-pd.Timedelta(days=1)))
    test_df = df.where(F.col('Date') >= test_begin)
    
    # Add a unique identifier for each column (e.g. Date -> Date_0, Date_1, etc ...)
    train_df = prepend_columns_index(train_df)
    val_df = prepend_columns_index(val_df)
    test_df = prepend_columns_index(test_df)
    
    # Convert each split from nxm -> nxm*(w+1), where w is the size of the window
    train_df = create_window_indexes(train_df, window_size)
    val_df = create_window_indexes(val_df, window_size)
    test_df = create_window_indexes(test_df, window_size)
    
    # Collect the processed datasets
    train_split.append(train_df)
    val_split.append(val_df)
    test_split.append(test_df)
  
  # Initialize the final datasets using the obtained df schema
  train_ds = spark.createDataFrame([], train_split[0].schema)
  val_ds = spark.createDataFrame([], val_split[0].schema)
  test_ds = spark.createDataFrame([], test_split[0].schema)
  
  # Collect all the dataframes into a single df for each data split
  for df in train_split: train_ds = train_ds.union(df)
  for df in val_split: val_ds = val_ds.union(df)
  for df in test_split: test_ds = test_ds.union(df)
  
  # Drop unused columns for the target variable and rename the target column as 'y'
  drop_cols = [col for col in test_ds.columns if should_drop(col, window_size)]
  train_ds = train_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  val_ds = val_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  test_ds = test_ds.drop(*drop_cols).withColumnRenamed(f'Adjusted Close_{window_size}', 'y')
  
  # Rename columns to make them compatible with Apache Parquet format
  target_cols = list(map(lambda x: '_'.join(re.split(' |/', x)), train_ds.columns))
  for c, target in zip(train_ds.columns, target_cols): train_ds = train_ds.withColumnRenamed(c, target)
  for c, target in zip(val_ds.columns, target_cols): val_ds = val_ds.withColumnRenamed(c, target)
  for c, target in zip(test_ds.columns, target_cols): test_ds = test_ds.withColumnRenamed(c, target)
  
  return train_ds, val_ds, test_ds

def prepend_columns_index(
  df: pyspark.sql.DataFrame,
  idx: int = 0
) -> pyspark.sql.DataFrame:
  ''' Utility function that returns a new dataframe where columns are preceeded by [idx]. '''
  for col in df.schema.names:
    # If this is not the first time this function is applied, simply edit the previous identifier
    new_col = f'{col}_{idx}' if idx == 0 else f'{col[:-2]}_{idx}'
    df = df.withColumnRenamed(col, new_col)
  return df

def create_window_indexes(df: pyspark.sql.DataFrame, window_size: int) -> pyspark.sql.DataFrame:
  ''' Apply self join multiple times to the input dataframe, return a new one of size (df.count(), len(df.columns)*window_size) '''
  joined_df = df
  for win_idx in range(1, window_size+1):
    joined_df = joined_df.join(
      prepend_columns_index(df, win_idx),
      F.col('id_0') == F.col(f'id_{win_idx}') - win_idx
    )
  return joined_df

#### Technical indicators

In [0]:
def add_sma(dfs: List[pyspark.sql.DataFrame], period: int = 10) -> None:
    ''' Computes the Simple Moving Average from a given dataframe. '''
    w = (Window().partitionBy("Adjusted Close").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
    #w = Window().partitionBy("Adjusted Close").orderBy(F.col("Date"))
    for i in tqdm(range(len(dfs)), desc='Adding SMA ...'):
        dfs[i] = dfs[i].withColumn(
          "SMA",
          F.avg("Adjusted Close").over(w)
        )

        
def add_rsi(dfs: List[pyspark.sql.DataFrame], period: int = 14) -> None:
    ''' Computes the Relative Strength Index from a given dataframe. 
        Formula available at https://en.wikipedia.org/wiki/Relative_strength_index.
        Also adds overbought and oversold when the RSI index hits 70 or 30.'''
    for j in tqdm(range(len(dfs)), desc='Adding RSI ...'):
        
      
        w = Window.partitionBy().orderBy("Date")

        dfs[j] = dfs[j].withColumn("prev_adj_close", F.lag(dfs[j]["Adjusted Close"]).over(w))
        dfs[j] = dfs[j].withColumn("current_gain", F.when(dfs[j]["Adjusted Close"] >= dfs[j]["prev_adj_close"],
                                                          dfs[j]["Adjusted Close"] - dfs[j]["prev_adj_close"])
                                                          .otherwise(0.0))
        dfs[j] = dfs[j].withColumn("current_loss", F.when(dfs[j]["prev_adj_close"] >= dfs[j]["Adjusted Close"],
                                                         dfs[j]["prev_adj_close"] - dfs[j]["Adjusted Close"])
                                                         .otherwise(0.0))
          
        w = (Window().partitionBy("current_gain").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
        dfs[j] = dfs[j].withColumn(
          "smmau",
          F.avg("current_gain").over(w)
        )
        
        w = (Window().partitionBy("current_loss").orderBy(F.col("Date").cast('long')).rangeBetween(-period*86400, 0))
        dfs[j] = dfs[j].withColumn(
          "smmad",
          F.avg("current_loss").over(w)
        )
        
        dfs[j] = dfs[j].withColumn("RSI", 100 - (100/(1+dfs[j]["smmau"]/dfs[j]["smmad"])))                        
        dfs[j] = dfs[j].withColumn("Overbought", F.when(dfs[j]["RSI"] >= 70, 1.0).otherwise(0.0))
        dfs[j] = dfs[j].withColumn("Oversold", F.when(dfs[j]["RSI"] <= 30, 1.0).otherwise(0.0))
        
        dfs[j] = dfs[j].drop("prev_adj_close", "current_gain", "current_loss", "smmau", "smmad")

In [0]:
print("Overview of the missing values in the key_stats dataframe\n")
key_stats_summary = missing_values_summary(key_stats_df)
key_stats_summary

### Missing values imputation

In [0]:
summary = prices_df_nan_summary(prices_dfs, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset before preprocessing (only columns with missing values are displayed)")

Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:03<00:24, 3.03s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:03<00:12, 1.81s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:04<00:08, 1.36s/it]Generating prices summary ...: 44%|████▍ | 4/9 [00:06<00:07, 1.58s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:07<00:05, 1.28s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:08<00:03, 1.16s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:09<00:02, 1.03s/it]Generating prices summary ...: 89%|████████▉ | 8/9 [00:09<00:00, 1.10it/s]Generating prices summary ...: 100%|██████████| 9/9 [00:10<00:00, 1.19it/s]Generating prices summary ...: 100%|██████████| 9/9 [00:10<00:00, 1.17s/it]
Out[23]:

For most of the above stocks with missing values, we noticed that they indeed exist up to a given time and after that no more data is available. It may due to a business failure, hence no more stocks will be exchanged from that moment on.

In [0]:
# Clear our input data from training NaN values
prices_dfs_new = [remove_trailing_nan(df,name) for df,name in tqdm(zip(prices_dfs, dfs_names), total=len(prices_dfs), desc='Removing trailing NaN values ...')]

# Remove INTH stock from our dataset since it contains many inactivity periods
#inth_idx = dfs_names.index('INTH') #TODO: uncomment
#del dfs_names[inth_idx] #TODO: uncomment
#del prices_dfs_new[inth_idx] #TODO: uncomment

summary = prices_df_nan_summary(prices_dfs_new, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset after preprocessing (only columns with missing values are displayed)")

Removing trailing NaN values ...: 0%| | 0/9 [00:00<?, ?it/s]Removing trailing NaN values ...: 11%|█ | 1/9 [00:03<00:31, 3.89s/it]Removing trailing NaN values ...: 22%|██▏ | 2/9 [00:06<00:20, 3.00s/it]Removing trailing NaN values ...: 33%|███▎ | 3/9 [00:08<00:15, 2.51s/it]Removing trailing NaN values ...: 44%|████▍ | 4/9 [00:10<00:11, 2.37s/it]Removing trailing NaN values ...: 56%|█████▌ | 5/9 [00:11<00:08, 2.07s/it]Removing trailing NaN values ...: 67%|██████▋ | 6/9 [00:14<00:06, 2.23s/it]Removing trailing NaN values ...: 78%|███████▊ | 7/9 [00:16<00:04, 2.19s/it]Removing trailing NaN values ...: 89%|████████▉ | 8/9 [00:18<00:02, 2.09s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:19<00:00, 1.89s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:19<00:00, 2.21s/it]
Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:01<00:11, 1.47s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:02<00:09, 1.40s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:04<00:09, 1.53s/it]Generating prices summary ...: 44%|████▍ | 4/9 [00:07<00:10, 2.15s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:09<00:07, 1.97s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:10<00:05, 1.84s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:12<00:03, 1.65s/it]Generating prices summary ...: 89%|████████▉ | 8/9 [00:13<00:01, 1.56s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:15<00:00, 1.77s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:15<00:00, 1.75s/it]
Out[24]:

At this point we use the fast forward imputation technique to fill-in missing values. Please note that in this case missing values are mostly due to holidays or periods when stocks are not exchanged.

### Building our new dataset

In [0]:
# Impute missing values in the prices dataset (i.e. fast-forward last valid values)
prices_dfs_new, key_stats_df_new = impute_missing_values(prices_dfs_new, key_stats_df)

# Merge the stock price dataset with fundamental data of the relative company
aggregate_dfs = merge_prices_fundamentals(prices_dfs_new, key_stats_df_new, dfs_names)

# Fill gaps from the original dataset
dfs = fill_missing_days(aggregate_dfs)
                  
# Add SMA indicator to each dataframe
add_sma(dfs)

# TODO: fix add_rsi and uncommented add_rsi and scale_features (which needs RSI)

# Add RSI indicator to each dataframe
#add_rsi(dfs)

# Scale numerical features
#scale_features(dfs)

# Create train/dev/test splits
train_ds, val_ds, test_ds = from_dfs(dfs)

Merging the datasets ...: 0%| | 0/56 [00:00<?, ?it/s]Merging the datasets ...: 18%|█▊ | 10/56 [00:00<00:01, 37.31it/s]Merging the datasets ...: 32%|███▏ | 18/56 [00:00<00:00, 43.04it/s]Merging the datasets ...: 41%|████ | 23/56 [00:00<00:00, 42.28it/s]Merging the datasets ...: 54%|█████▎ | 30/56 [00:00<00:00, 47.64it/s]Merging the datasets ...: 79%|███████▊ | 44/56 [00:00<00:00, 64.28it/s]Merging the datasets ...: 100%|██████████| 56/56 [00:00<00:00, 70.64it/s]Merging the datasets ...: 100%|██████████| 56/56 [00:00<00:00, 58.10it/s]
Filling missing days ...: 0%| | 0/6 [00:00<?, ?it/s]Filling missing days ...: 17%|█▋ | 1/6 [00:05<00:26, 5.30s/it]Filling missing days ...: 33%|███▎ | 2/6 [00:09<00:19, 4.90s/it]Filling missing days ...: 50%|█████ | 3/6 [00:12<00:12, 4.06s/it]Filling missing days ...: 67%|██████▋ | 4/6 [00:15<00:07, 3.51s/it]Filling missing days ...: 83%|████████▎ | 5/6 [00:19<00:03, 3.64s/it]Filling missing days ...: 100%|██████████| 6/6 [00:21<00:00, 3.24s/it]Filling missing days ...: 100%|██████████| 6/6 [00:21<00:00, 3.66s/it]
Adding SMA ...: 0%| | 0/6 [00:00<?, ?it/s]Adding SMA ...: 33%|███▎ | 2/6 [00:00<00:00, 14.75it/s]Adding SMA ...: 83%|████████▎ | 5/6 [00:00<00:00, 22.35it/s]Adding SMA ...: 100%|██████████| 6/6 [00:00<00:00, 21.98it/s]
 0%| | 0/6 [00:00<?, ?it/s] 17%|█▋ | 1/6 [00:05<00:28, 5.63s/it] 33%|███▎ | 2/6 [00:09<00:18, 4.54s/it] 50%|█████ | 3/6 [00:12<00:12, 4.08s/it] 67%|██████▋ | 4/6 [00:16<00:07, 3.95s/it] 83%|████████▎ | 5/6 [00:21<00:04, 4.10s/it]100%|██████████| 6/6 [00:24<00:00, 4.00s/it]100%|██████████| 6/6 [00:24<00:00, 4.14s/it]

In [0]:
'''
Work in progresss ...
'''

converter_train = make_spark_converter(train_ds)
with converter_train.make_torch_dataloader(batch_size=1024) as train_dataloader:
  train_dataloader_iter = iter(train_dataloader)
  x = next(train_dataloader_iter)
  print(x['Price_0'].shape)

The median size 143931 B (< 50 MB) of the parquet files is too small. Total size: 724238 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:/dbfs/tmp/petastorm/cache/20210513204349-appid-local-1620935412485-7f5f3109-b529-4461-963b-00ef1c61c5aa/part-00009-tid-3308809889305446372-2d8f786d-a6d3-4a62-a78c-078f11aac402-963-1-c000.parquet, ...
torch.Size([1024])

In [0]:
'''
Work in progresss ...
'''

unique_cols = set(c[:-2] for c in train_ds.columns if len(c) > 2)
ls = []
for k,v in x.items():
  if 'Price_' == k[:-1]:
    ls.append(v)
torch.cat(ls).shape

Out[105]: torch.Size([3072])

In [0]:
train_ds

Out[109]: DataFrame[Price_0: double, DE_Ratio_0: double, Trailing_P_E_0: double, Price_Sales_0: double, Price_Book_0: double, Profit_Margin_0: double, Operating_Margin_0: double, Return_on_Assets_0: double, Return_on_Equity_0: double, Revenue_Per_Share_0: double, Market_Cap_0: double, Enterprise_Value_0: double, PEG_Ratio_0: double, Enterprise_Value_Revenue_0: double, Enterprise_Value_EBITDA_0: double, Revenue_0: double, Gross_Profit_0: double, EBITDA_0: double, Net_Income_Avl_to_Common__0: double, Diluted_EPS_0: double, Earnings_Growth_0: double, Revenue_Growth_0: double, Total_Cash_0: double, Total_Cash_Per_Share_0: double, Total_Debt_0: double, Current_Ratio_0: double, Book_Value_Per_Share_0: double, Cash_Flow_0: double, Beta_0: double, Volume_0: double, Adjusted_Close_0: double, SMA_0: double, Price_1: double, DE_Ratio_1: double, Trailing_P_E_1: double, Price_Sales_1: double, Price_Book_1: double, Profit_Margin_1: double, Operating_Margin_1: double, Return_on_Assets_1: double, Return_on_Equity_1: double, Revenue_Per_Share_1: double, Market_Cap_1: double, Enterprise_Value_1: double, PEG_Ratio_1: double, Enterprise_Value_Revenue_1: double, Enterprise_Value_EBITDA_1: double, Revenue_1: double, Gross_Profit_1: double, EBITDA_1: double, Net_Income_Avl_to_Common__1: double, Diluted_EPS_1: double, Earnings_Growth_1: double, Revenue_Growth_1: double, Total_Cash_1: double, Total_Cash_Per_Share_1: double, Total_Debt_1: double, Current_Ratio_1: double, Book_Value_Per_Share_1: double, Cash_Flow_1: double, Beta_1: double, Volume_1: double, Adjusted_Close_1: double, SMA_1: double, Price_2: double, DE_Ratio_2: double, Trailing_P_E_2: double, Price_Sales_2: double, Price_Book_2: double, Profit_Margin_2: double, Operating_Margin_2: double, Return_on_Assets_2: double, Return_on_Equity_2: double, Revenue_Per_Share_2: double, Market_Cap_2: double, Enterprise_Value_2: double, PEG_Ratio_2: double, Enterprise_Value_Revenue_2: double, Enterprise_Value_EBITDA_2: double, Revenue_2: double, Gross_Profit_2: double, EBITDA_2: double, Net_Income_Avl_to_Common__2: double, Diluted_EPS_2: double, Earnings_Growth_2: double, Revenue_Growth_2: double, Total_Cash_2: double, Total_Cash_Per_Share_2: double, Total_Debt_2: double, Current_Ratio_2: double, Book_Value_Per_Share_2: double, Cash_Flow_2: double, Beta_2: double, Volume_2: double, Adjusted_Close_2: double, SMA_2: double, y: double]